In [1]:
import numpy as np

In [2]:
#Generate some data to play with
np.random.seed(42)
pp = 100
n = 100000
n_samples, n_features =n , pp
data_X = np.random.randn(n_samples, n_features)
coef = 3 * np.random.randn(n_features)
inds = np.arange(n_features)
np.random.shuffle(inds)
coef[inds[5:]] = 0  # sparsify coef
data_y = np.dot(data_X, coef)
# add noise
data_y += 0.02 * np.random.normal((n_samples,))

In [3]:
#Import packages
import numba
from numba import jit

In [4]:
residual = np.empty([n])
result = np.empty([n])
beta = np.random.randn(n_features)
k = 1
data = data_X
y = data_y
#Soft threshold operator
def S(z, gamma):
    if np.abs(z) - gamma > 0:
        return np.sign(z)*(np.abs(z) - gamma)
    else:
        return 0

In [5]:
@jit( cache=True)
def myUpdate(residual, result, beta, data, y, n, l1_ratio, alpha,z):
    for k in range(pp):
        for j in range(n):
            for i in range(pp):
                if i != k:
                    result[j] += beta[i]*data[j,i]
            residual[j] = y[j]-result[j]
        for j in range(n): 
            z+=(1/n)* data[j,k]*residual[j]
        beta[k]=S(z, alpha*l1_ratio)/(1+alpha*(1-l1_ratio))
    return beta

In [ ]:
myUpdate(residual, result, beta, data, y,n,1,.5,1)